In [16]:
import pandas as pd
import numpy as np
import xgboost as xgb
import hyperopt as hpt
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings("ignore")
import time
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.grid_search import RandomizedSearchCV
import collections
import matplotlib
%matplotlib inline

In [2]:
load_data = pd.read_csv('/home/gkaiafas/datasets/Load_Protergia/0a05916b-d667-497f-b78c-749536429dbd.csv')
load_data.columns=['Time', 'kwh']
load_data['Time'] = pd.to_datetime(load_data['Time'])
load_data.head()
#### we sort the dataframe based on Time
data = load_data.sort('Time')

In [3]:
data['day_of_the_year'] = data['Time'].map(lambda x: x.dayofyear)
data['hour'] = data['Time'].map(lambda x: x.hour)
data['weekday'] = data['Time'].map(lambda x: x.weekday())
data['quarter'] = data['Time'].map(lambda x: x.quarter)
data['year'] = data['Time'].map(lambda x: x.year)

In [4]:
print len(data.kwh[data['kwh']==0])
print len(data.kwh[data['kwh']<0])
print len(data.kwh[data['kwh']==max(data.kwh)])
print max(data.kwh[data.kwh!=max(data.kwh)])
print max(data.kwh)

8038
0
4
131.8
10000.0


In [5]:
data.kwh[data.kwh==0.0] = min(data.kwh[(data.kwh>0.0) & (data.kwh!=max(data.kwh))])
data.ix[data.kwh[data.kwh==max(data.kwh)].index] =  data.kwh.ix[data.kwh[data.kwh==max(data.kwh)].index[0]-1]

#####  We want to know if a day has 96 measurements, so with the following group_by we investigate it

In [6]:
t = data.groupby(['day_of_the_year','year'])
groups = (dict(list(t)))
groups = collections.OrderedDict(sorted(groups.items()))
# print groups[1].iloc[0]
# print groups[365].iloc[0]
print len(groups)
for i in groups.keys():
    if len(groups[i]) != 96:
        del groups[i]
print len(groups)

1245
1243


######  After we check we reconstruct the dataframe in order to continue our super-slice

In [7]:
list_of_dataframes = [groups[i] for i in groups.keys()]
data = pd.concat(list_of_dataframes)

In [8]:
quarter_slices_dict = dict(list(data.groupby(['quarter'])))

In [9]:
weekdays_hours_quarter1 = dict(list(quarter_slices_dict[quarter_slices_dict.keys()[0]].groupby(['weekday'])))
weekdays_hours_quarter2 = dict(list(quarter_slices_dict[quarter_slices_dict.keys()[1]].groupby(['weekday'])))
weekdays_hours_quarter3 = dict(list(quarter_slices_dict[quarter_slices_dict.keys()[2]].groupby(['weekday'])))
weekdays_hours_quarter4 = dict(list(quarter_slices_dict[quarter_slices_dict.keys()[3]].groupby(['weekday'])))

In [10]:
indices = list(np.arange(1,4600, 96))
weekdays_hours_quarter1[0].iloc[indices]

,Time,kwh,day_of_the_year,hour,weekday,quarter,year
19393,2012-01-02 00:15:00,31.6,2,0,0,1,2012
81889,2015-01-05 00:15:00,24.4,5,0,0,1,2015
55681,2013-01-07 00:15:00,2.2,7,0,0,1,2013
18145,2012-01-09 00:15:00,25.6,9,0,0,1,2012
81409,2015-01-12 00:15:00,29.4,12,0,0,1,2015
54529,2013-01-14 00:15:00,2.2,14,0,0,1,2013
18721,2012-01-16 00:15:00,15.0,16,0,0,1,2012
81601,2015-01-19 00:15:00,45.4,19,0,0,1,2015
53281,2013-01-21 00:15:00,1.8,21,0,0,1,2013
19777,2012-01-23 00:15:00,10.0,23,0,0,1,2012


In [11]:
# With this value we get all the unique days & year in the dataframe 'weekdays_hours_quarter1[0]' which has all the hours
# for weekday==0
p = 0
y = []
# df = pd.DataFrame([1,2,3,4,5,6], columns=list('ABDFGR'))
for i in range(0,len(weekdays_hours_quarter1[0]),96):
    y.append(np.ravel(weekdays_hours_quarter1[0][['day_of_the_year','year']][p*96:(p*96)+1].values))
    p += 1
t = pd.DataFrame.from_records(y)
t.columns = ['year_of_the_day', 'year']
print t.head()
print t.tail()
print ("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
# Another efficient way to reveal all the unique days is the following
indices = list(np.arange(1,4600, 96))
t1 = weekdays_hours_quarter1[0].iloc[indices][['day_of_the_year', 'year']]
print t1.head()
print t1.tail()

   year_of_the_day  year
0                2  2012
1                5  2015
2                7  2013
3                9  2012
4               12  2015
    year_of_the_day  year
43               83  2014
44               84  2013
45               86  2012
46               89  2015
47               90  2014
++++++++++++++++++++++++++++++++++++++++++++++++++++++
       day_of_the_year  year
19393                2  2012
81889                5  2015
55681                7  2013
18145                9  2012
81409               12  2015
        day_of_the_year  year
108385               83  2014
59617                84  2013
24865                86  2012
86881                89  2015
110593               90  2014


#####  We see that the previous approaches lead to eqaul results

### We want  to reveal all the quarter consumption for the previous day in order to add it in the feature space. The following code does it just for the dataframe of weekdays==0 which we calculated before.

In [12]:
# We take all the values of the previous dataframe and we create a list with the previous days writings on each quarter
previous_1_day = []
for i in range(len(t)):
    dayy = t.year_of_the_day.iloc[i]
    yearr = t.year.iloc[i]
    if len(data[(data.year==yearr) & (data.day_of_the_year==dayy-1)]) == 0:
            for iii in range(96):
                previous_1_day.append(np.nan)
    else:
        for ii in range(96):
            previous_1_day.append(data[(data.year==yearr) & (data.day_of_the_year==dayy-1)].iloc[ii]['kwh'])
previous_1_day = pd.Series(previous_1_day)
previous_1_day.index = weekdays_hours_quarter1[0].index
weekdays_hours_quarter1[0]['1_daybefore'] = pd.Series(previous_1_day)
weekdays_hours_quarter1[0].head()

,Time,kwh,day_of_the_year,hour,weekday,quarter,year,1_daybefore
19392,2012-01-02 00:00:00,9.0,2,0,0,1,2012,26.0
19393,2012-01-02 00:15:00,31.6,2,0,0,1,2012,8.2
19394,2012-01-02 00:30:00,28.2,2,0,0,1,2012,8.0
19395,2012-01-02 00:45:00,36.4,2,0,0,1,2012,8.0
19396,2012-01-02 01:00:00,9.8,2,1,0,1,2012,22.4


In [13]:
print len(pd.DataFrame(previous_1_day))
print "=====================++++=+========"
print len(weekdays_hours_quarter1[0])

4608
=====================++++=+========
4608


### We make a function that generalizes the previous for a given weekday and how many days we want to look back, for the dataframe  with quarter==1

In [14]:
def get_previous_i_days_consumptions_for_weekday_J_quarter_1(M, J, Q):  
    if Q==1:
        sliced_dataframe = weekdays_hours_quarter1
    elif Q==2:
            sliced_dataframe = weekdays_hours_quarter2
    elif Q==3:
                sliced_dataframe = weekdays_hours_quarter3
    elif Q==4:
                    sliced_dataframe = weekdays_hours_quarter4
    else:
        print "Give an integer value between 1 and 4"
            
    p = 0
    y = []
# df = pd.DataFrame([1,2,3,4,5,6], columns=list('ABDFGR'))
    for i in range(0,len(sliced_dataframe[J]),96):
        y.append(np.ravel(sliced_dataframe[J][['day_of_the_year','year']][p*96:(p*96)+1].values))
        p += 1
    t = pd.DataFrame.from_records(y)
    t.columns = ['year_of_the_day', 'year']
    previous_i_day = []
    for i in range(len(t)):
            dayy = t.year_of_the_day.iloc[i]
            yearr = t.year.iloc[i]
            if len(data[(data.year==yearr) & (data.day_of_the_year==dayy - M)]) == 0:
                    for iii in range(96):
                        previous_i_day.append(np.nan)
            else:
                for ii in range(96):
                    previous_i_day.append(data[(data.year==yearr) & (data.day_of_the_year==dayy - M)].iloc[ii]['kwh'])
    previous_i_day = pd.Series(previous_i_day)
    previous_i_day.index = sliced_dataframe[J].index
    return previous_i_day

##### Just for confiramtion we proove that for i=0 which means, look back 0 days from the given day we MUST take as output from the function exactly the same kwh consumption which happens below.

In [15]:
sum(get_previous_i_days_consumptions_for_weekday_J_quarter_1(0,6,2).values-weekdays_hours_quarter2[6]['kwh'].values)

0.0

#### As next step for each weekday in the corresponding dataframe we will add some features such as the consumption for the previous day, the pre-previous day until 1 wek before